# Building a handwritten digit recognition system using MNIST dataset

---




## Problem


Handwritten digit recognition is a common problem in a field of Optical Character Recognition `(OCR)`, and it has many practical applications, such as digitizing documents, automating data entry, or processing handwritten forms, such as invoices or surveys. In this project, we will develop a system that can recognize handwritten digits from images or digits written on a screen by using machine learning technology. This system can be extended to perform some basic calculations based on the recognized digits.
The aim of this project are:
- Build a system that can accurately recognize handwritten digits `(0–9)` from input images.
- Convert handwritten input into machine-readable format and perform basic calculations on them such as: addition, subtraction, multiplication and division.

---

## Some main tasks:
- `Data colection`: In this project, you use the MNIST dataset, which consists of 60,000 training images and 10,000 testing images of handwritten digits. Each image is 28x28 pixels and labeled with the correct digit.
- `Image Preprocessing`: Normalize the image data (e.g., convert pixel values to a scale from 0 to 1), resize all images to the same dimensions, and apply noise reduction to improve recognition accuracy.
- `Model Development`: We will study how to build and train a model using an advanced technique that separates different classes of digits in machine learning which is `Support Vector Machine (SVM)` algorithm.
- `Evaluation`: Test the model’s performance using the test dataset and assess its accuracy using metrics like accuracy, precision, recall, and F1 score.
- `Application`: Implement a user interface where users can upload or draw digits on screen, and the system will return the predicted digit based on the trained model. Your system should also be able to work with basic calculations.

# 📚 | Importing libraries

In [ ]:
import os

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Processing data
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# library for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Load model
from model.SVM import SVC

# Load datasets
from tensorflow.keras import datasets

# library for data evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
import cv2

# 📖 | Meta Data
`MNIST dataset`: This is a dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. More info can be found at the [MNIST homepage](https://yann.lecun.com/exdb/mnist).

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
df_digit_draw = pd.read_csv("./dataset/digit_dataset.csv")
df_digit_draw.head()

In [ ]:
df_symboils = pd.read_csv('./dataset/symbols_dataset_new.csv')
df_symboils.head()

# 📊 | Data preprocessing

In [ ]:
X_train_draw = df_digit_draw.drop("label", axis = 1)
X_train_draw.head()

In [ ]:
y_train_draw = df_digit_draw["label"]
y_train_draw.head()

In [ ]:
X_train_draw_flatten = X_train_draw.astype('float32') / 255
X_train_draw_flatten = X_train_draw_flatten.to_numpy()
X_train_draw_resize = X_train_draw_flatten.reshape(-1, 28 * 28)
print(X_train_draw_resize.shape)
X_train_draw_resize = pd.DataFrame(X_train_draw_resize)
print(X_train_draw_resize.head())

In [ ]:
df_digit_draw = pd.concat([X_train_draw_resize, y_train_draw], axis = 1)
df_digit_draw.head()

In [ ]:
X_train_flatten = X_train.astype('float32') / 255
X_test_flatten = X_test.astype('float32') / 255

In [ ]:
X_train_resize = X_train_flatten.reshape(-1, 28 * 28)
X_test_resize = X_test_flatten.reshape(-1, 28 * 28)

print(X_train_resize.shape, X_test_resize.shape)

In [ ]:
df_X_train = pd.DataFrame(X_train_resize)
df_X_test = pd.DataFrame(X_test_resize)
df_y_train = pd.DataFrame(y_train)
df_y_test = pd.DataFrame(y_test)

df_X_train.head()

In [ ]:
df_y_train = df_y_train.rename(columns = {0: 'label'})

In [ ]:
df_digit = pd.concat([df_X_train, df_y_train], axis = 1)
df_digit.head()

In [ ]:
new_column_names = ['pixel' + str(i) for i in range(784)] + ['label']
df_digit_draw.columns = new_column_names
df_digit_draw.head()

## 📊 | Processing Data Symbols

In [ ]:
df_symboils.info()

In [ ]:
df_symboils.columns = new_column_names
df_symboils.head()

In [ ]:
df_concat = pd.concat([df_digit_draw, df_symboils])
df_concat = df_concat.reset_index(drop = True)

df_concat.tail()

In [ ]:
# Assuming that the label for the symbols are as follows:
label_mapping = {'+': 10, '-': 11, '*': 12, '/': 13}

# Replace label in the 'label' column using the mapping
df_concat['label'] = df_concat['label'].replace(label_mapping)

# Now your Dataframe has labels 10, 11, 12, 13 for +, -, *, / respectively
df_concat.tail()

In [ ]:
X = df_concat.drop(columns = 'label')
y = df_concat['label']

X.shape, y.shape

In [ ]:
ramdom = RandomUnderSampler(random_state = 42)
X_random, y_random = ramdom.fit_resample(X, y)

print(y.value_counts())
print(y_random.value_counts())

In [ ]:
# Imblearn dataset
smote = SMOTE(random_state = 42)
X_smote, y_smote = smote.fit_resample(X, y)

print(y.value_counts())
print(y_smote.value_counts())

# 🪄 | Data visualization

In [ ]:
# Visualizing the digits and symbols
plt.figure(figsize = (14, 12))
for digit_image in range(0, 30):
    plt.subplot(7, 10, digit_image + 1)
    grid_data = X.iloc[-digit_image].values.reshape(28, 28)
    plt.imshow(grid_data, interpolation = 'None', cmap = 'gray')
    plt.xticks([])
    plt.yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Visualizing the number of class and counts in the datasets
def class_distribution(y):
    sns.countplot(x = y, palette = 'Set1')
    plt.xlabel('Class')
    plt.ylabel('Counts')
    plt.title('Class Distribution')
    plt.show()


class_distribution(y_random)

# ☣ | Building model SVM

In [ ]:
#Splitting the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X_random, y_random, test_size = 0.2, random_state = 42)

# Print the shape of the data
print(f'X_train = {X_train.shape}, y_train = {y_train.shape}, X_test = {X_test.shape}, y_test = {y_test.shape}')

In [ ]:
# Visualizing the number of class and counts in the datasets
class_distribution(y_train)

In [ ]:
# Visualizing the number of class and counts in the datasets
class_distribution(y_test)

In [ ]:
# Convert the data into numpy arrays
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()
X_test_array = X_test.to_numpy()
y_test_array = y_test.to_numpy()

# Print the shape of the data
X_train_array[:5], y_train_array[:5]

In [ ]:
# Create the model
# The model is created with the following hyperparameters:
# Learning rate = 0.001
# Number of iterations = 100
# Lambda parameter = 0.0001
# The model is trained on the training data
svc = SVC(learning_rate = 0.001, no_of_iterations = 100, lambda_parameter = 0.0001)

# Fit the model
svc.fit(X_train_array, y_train_array)

In [ ]:
# Predict the model
y_predict = svc.predict(X_test_array)

# Print the predicted values
print(f'Predict values: {y_predict[:5]}')

# Print the actual values
print(f'Actual values: {y_test_array[:5]}')

# 📊 | Model evaluation

In [ ]:
# Calculate the accuracy of the model
test_accuracy = accuracy_score(y_test_array, y_predict)
print(f'Test Accuracy: {test_accuracy * 100:.3f}%')

In [ ]:
# Confusion matrix
confusion = confusion_matrix(y_test_array, y_predict)

# Plot the confusion matrix
sns.heatmap(confusion, annot = True, fmt = 'd', cmap = 'Blues')
plt.show()

In [ ]:
# Classification report
class_wine = classification_report(y_test_array, y_predict)
print(class_wine)

# 📸 | Predicting the image

In [ ]:
image = cv2.imread('D:\Graduation_Project\dataset\digit_test\img_184.jpg')

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred_image = cv2.GaussianBlur(gray_image, (3, 3), 0)

resize_image = cv2.resize(blurred_image, (28, 28))

plt.imshow(resize_image, cmap = 'gray')
plt.show()

flatten_image = resize_image.astype('float32') / 255
flatten_image = flatten_image.reshape(1, -1)

prediction = svc.predict(flatten_image)
print(prediction)

# 💾 | Save the model

In [ ]:
joblib.dump(svc, r"D:\Graduation_Project\model\svm_digit_classifier_version3.pkl")